<a href="https://colab.research.google.com/github/Ervinoreo/Deep_Learning/blob/main/cnn(cifar10).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as py
import numpy as np

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

num_epochs = 100
batch_size = 4
learning_rate = 0.001

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root = 'data', train = True, download = True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(root = 'data', train = False, download = True, transform = transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size, shuffle = False)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')



100%|██████████| 170498071/170498071 [00:02<00:00, 75734756.34it/s]


Extracting data/cifar-10-python.tar.gz to data
Files already downloaded and verified


In [3]:
class ConvNet(nn.Module):
  def __init__(self):
    super(ConvNet, self).__init__()
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)


  def forward(self, x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    x = x.view(-1,16*5*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


model = ConvNet().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

n_total_step = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.to(device)
    labels = labels.to(device)

    outputs = model(images)
    loss = criterion(outputs, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i + 1) % 2000 == 0:
      print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_step}], Loss: {loss.item():.4f}')



with torch.no_grad():
  n_correct = 0
  n_samples = 0
  n_class_correct = [0 for i in range(10)]
  n_class_samples = [0 for i in range(10)]

  for images, labels in test_loader:
    images = images.to(device)
    labels = labels.to(device)
    outputs = model(images)

    _, predicted = torch.max(outputs, 1)
    n_samples += labels.size(0)
    n_correct += (predicted == labels).sum().item()

    for i in range(batch_size):
      label = labels[i]
      pred = predicted[i]
      if (label == pred):
        n_class_correct[label] += 1
      n_class_samples[label] += 1

  acc = 100.0 * n_correct / n_samples
  print(f'Accuracy of the network {acc} %')

  for i in range(10):
    acc = 100.0 * n_class_correct[i] / n_class_samples[i]
    print(f'Accuracy of {classes[i]}: {acc}%')



Epoch [1/100], Step [2000/12500], Loss: 2.3176
Epoch [1/100], Step [4000/12500], Loss: 2.3038
Epoch [1/100], Step [6000/12500], Loss: 2.2818
Epoch [1/100], Step [8000/12500], Loss: 2.2952
Epoch [1/100], Step [10000/12500], Loss: 2.2859
Epoch [1/100], Step [12000/12500], Loss: 2.3981
Epoch [2/100], Step [2000/12500], Loss: 2.6761
Epoch [2/100], Step [4000/12500], Loss: 1.9823
Epoch [2/100], Step [6000/12500], Loss: 2.2402
Epoch [2/100], Step [8000/12500], Loss: 1.5190
Epoch [2/100], Step [10000/12500], Loss: 1.9361
Epoch [2/100], Step [12000/12500], Loss: 1.0975
Epoch [3/100], Step [2000/12500], Loss: 1.1448
Epoch [3/100], Step [4000/12500], Loss: 1.7694
Epoch [3/100], Step [6000/12500], Loss: 2.1333
Epoch [3/100], Step [8000/12500], Loss: 1.4434
Epoch [3/100], Step [10000/12500], Loss: 2.4362
Epoch [3/100], Step [12000/12500], Loss: 1.4172
Epoch [4/100], Step [2000/12500], Loss: 2.0119
Epoch [4/100], Step [4000/12500], Loss: 1.8299
Epoch [4/100], Step [6000/12500], Loss: 0.6452
Epoch [